In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.8 MB/s eta 0:00:00


In [3]:
import pulp as plp
from pulp import *
import numpy as np
import pandas as pd

In [5]:
# Set problem

prob = plp.LpProblem("United_Airlines_flight", LpMaximize)

<li> Create a PuLP problem object named "United_Airlines_flight".
<li> set type of problem as maximisation. <br>
<b><center>Maximize Profit=Total Revenue−Total Costs

<center><h3> FLEET (AIRCRAFTS) DATA

In [6]:
# Aicrafts
aircrafts = ["Boeing 787","Airbus 320","Boeing 737","Boeing 767","Boeing 777"]

In [8]:
# Availability (no of aircraft of each type that are available)
aircrafts_avail = [48, 97, 136, 54, 94]

In [10]:
# seating capacity for each aircraft
seat = [300, 150, 160, 260, 320]

In [11]:
# Cost per hour for each aircraft
cost_flight_hour = [5310, 2840, 3010, 4980, 7220]

In [14]:
'''
#summarising
0- Boeing 787 - PAX = 300, OPC = 5310, Availability = 48
1- Airbus 320 - PAX = 150, OPC = 2840, Availability = 97
2- Boeing 737 - PAX = 160, OPC = 3010, Availability = 136
3- Boeing 767 - PAX = 260, OPC = 4980, Availability = 54
4- Boeing 777 - PAX = 320, OPC = 7220, Availability = 94
'''

'\n#summarising\n0- Boeing 787 - PAX = 300, OPC = 5310, Availability = 48\n1- Airbus 320 - PAX = 150, OPC = 2840, Availability = 97\n2- Boeing 737 - PAX = 160, OPC = 3010, Availability = 136\n3- Boeing 767 - PAX = 260, OPC = 4980, Availability = 54\n4- Boeing 777 - PAX = 320, OPC = 7220, Availability = 94\n'

<center><h3> FLIGHT DATA

In [21]:
# Airports (Hubs)
airports = ['ORD','DEN','IAH','LAX','EWR','SFO','IAD']
# #Hubs
# 0- Chicago–O'Hare - ORD
# 1- Denver - DEN
# 2- Houston–Intercontinental - IAH
# 3- Los Angeles - LAX
# 4- Newark - EWR
# 5- San Francisco - SFO
# 6- Washington–Dulles - IAD

In [16]:
# Fares matrix (each index is the airports) (diagonal=0)
#$x,000 unit
fare = np.array([
[0  ,	47 ,	81 ,	50 ,	136,	116,	123],
[51 ,	0  ,	64 ,	47 ,	137,	70 ,	156],
[79 ,	79 ,	0  ,	94 ,	68 ,	215,	189],
[45 ,	59 ,	94 ,	0  ,	129,	64 ,	156],
[125,	125,	79 ,	139,	0  ,	124,	149],
[110,	70 ,	209,	59 ,	139,	0  ,	307],
[119,	149,	279,	125,	139,	299,	0  ]])

In [18]:
# Passenger Demand
demand = np.array([
[0   ,	1900,	650 ,	1100,	2350,	1000,	400 ],
[1800,	0   ,	600 ,	1500,	1000,	1000,	500 ],
[750 ,	550 ,	0   ,	1300,	1350,	500,	950 ],
[1200,	1600,	1200,	0   ,	2100,	3000,	1950],
[2450,	900 ,	1250,	2000,	0   ,	2150,	1900],
[1100,	1200,	450 ,	2800,	2200,	0   ,	750 ],
[450 ,	550 ,	900 ,	1900,	1850,	950 ,	0   ]])

In [19]:
# Time of flight in hours
time_flight = np.array([
[0   ,	2.55,	2.5 ,	4.3 ,	2.2 ,	4.5 ,	1.75],
[2.55,	0   ,	2.4 ,	2.3 ,	3.6 ,	2.55,	3.25],
[2.5 ,	2.4 ,	0   ,	3.4 ,	3.5 ,	4.45,	2.8 ],
[4.3 ,	2.3 ,	3.4 ,	0   ,	4.9 ,	1.4 ,	4.75],
[2.2 ,	3.6 ,	3.5 ,	4.9 ,	0   ,	6.25,	1.25],
[4.5 ,	2.55,	4.45,	1.4 ,	6.25,	0   ,	5   ],
[1.75,	3.25,	2.8 ,	4.75,	1.25,	5   ,	0   ]])

In [20]:
# Frequency/ no of flights per day
frequency = np.array([
[0  ,	10 ,	5  ,	6  ,	12 ,	4  ,	4  ],
[9  ,	0  ,	5  ,	5  ,	6  ,	6  ,	4  ],
[4  ,	4  ,	0  ,	10 ,	10 ,	5  ,	5  ],
[5  ,	4  ,	9  ,	0  ,	15 ,	15 ,	10 ],
[11 ,	5  ,	9  ,	14 ,	0  ,	12 ,	11 ],
[3  ,	5  ,	4  ,	14 ,	11 ,	0  ,	5  ],
[3  ,	3  ,	4  ,	9  ,	10 ,	4  ,	0  ]])

<h4><b> declaring decision variables

In [22]:
# dec Variables

x = {
    i: {
        j: {
            k: plp.LpVariable(
                name="{} to {},type {}".format(airports[i],airports[j],aircrafts[k]),   #human-readable variable names for solver logs and debugging
                lowBound=0,  #non negativity, no of planes assigned cant be negative, can be 0
                cat=plp.LpInteger  #for integer values / no of planes
            )
            for k in range(len(aircrafts))
        }
        for j in range(len(airports))
    }
    for i in range(len(airports))
}
x # would make 210 variables : 7*6*5

{0: {0: {0: ORD_to_ORD,type_Boeing_787,
   1: ORD_to_ORD,type_Airbus_320,
   2: ORD_to_ORD,type_Boeing_737,
   3: ORD_to_ORD,type_Boeing_767,
   4: ORD_to_ORD,type_Boeing_777},
  1: {0: ORD_to_DEN,type_Boeing_787,
   1: ORD_to_DEN,type_Airbus_320,
   2: ORD_to_DEN,type_Boeing_737,
   3: ORD_to_DEN,type_Boeing_767,
   4: ORD_to_DEN,type_Boeing_777},
  2: {0: ORD_to_IAH,type_Boeing_787,
   1: ORD_to_IAH,type_Airbus_320,
   2: ORD_to_IAH,type_Boeing_737,
   3: ORD_to_IAH,type_Boeing_767,
   4: ORD_to_IAH,type_Boeing_777},
  3: {0: ORD_to_LAX,type_Boeing_787,
   1: ORD_to_LAX,type_Airbus_320,
   2: ORD_to_LAX,type_Boeing_737,
   3: ORD_to_LAX,type_Boeing_767,
   4: ORD_to_LAX,type_Boeing_777},
  4: {0: ORD_to_EWR,type_Boeing_787,
   1: ORD_to_EWR,type_Airbus_320,
   2: ORD_to_EWR,type_Boeing_737,
   3: ORD_to_EWR,type_Boeing_767,
   4: ORD_to_EWR,type_Boeing_777},
  5: {0: ORD_to_SFO,type_Boeing_787,
   1: ORD_to_SFO,type_Airbus_320,
   2: ORD_to_SFO,type_Boeing_737,
   3: ORD_to_SFO,type_

In [23]:
# Revenue = Σ Fare ij * Passanger demand ij
#sanity check to get an estimate of totale per day reveune if all demand is met

Revenue = pulp.lpSum(fare[i][j] * demand[i][j]
                       for i in range(len(time_flight))
                       for j in range(len(fare)))

print("Revenue for Airlines is: $",Revenue)

Revenue for Airlines is: $ 6310300.0


<center> <h2> Objective Function

In [24]:

# Profit (Z) = Revenue - Cost

prob += Revenue - pulp.lpSum(x[i][j][k] * cost_flight_hour[k] * time_flight[i][j]
                       for j in range(len(airports))
                       for k in range(len(cost_flight_hour))
                       for i in range(len(time_flight)))

**Objective Function**

$$
\text{Maximize: Profit} = \sum_{i} \sum_{j} \sum_{k}
\Big[ (\text{fare}_{ij} \times \text{seats}_{k})
- \left( \frac{\text{cost}}{\text{hr}}_{k} \times \text{time}_{ij} \right) \Big]
\times X_{i,j,k}
$$

**Where:**
- \(i\) = origin airport index  
- \(j\) = destination airport index  
- \(k\) = aircraft type index  
- \(\text{fare}_{ij}\) = ticket price per passenger on route \(i \to j\)  
- \(\text{seats}_{k}\) = seat capacity of aircraft type \(k\)  
- \(\frac{\text{cost}}{\text{hr}}_{k}\) = operating cost per flight-hour for aircraft type \(k\)  
- \(\text{time}_{ij}\) = block time (in hours) for route \(i \to j\)  
- \(X_{i,j,k}\) = number of planes of type \(k\) assigned to route \(i \to j\)  


In [30]:
#constraints
#C1

# No flight from/to the same departure point / avoid self loops

for i in range(len(airports)):
    for k in range(len(aircrafts)):
        prob += (x[i][i][k] == 0), 'No flights to and from {} (Flight type {})'.format(airports[i],aircrafts[k])
        #giving name to the constraint, can skip

In [31]:
#C2

# We need to meet the demand of passengers moving from airport to airport

# Σ(X ijk)(Sk) ≥ (demand from i to j)

for i in range(len(airports)):
    for j in range(len(airports)):
        prob += pulp.lpSum(seat[k] * x[i][j][k] for k in range(len(aircrafts))) >= demand[i][j],'Demand Constraint: {} to {}'.format(airports[i],airports[j])

**Demand Satisfaction Constraint**

$$
\sum_{k} \text{seats}_{k} \cdot X_{i,j,k} \;\ge\; \text{demand}_{i,j}
$$

Where:
- \(i\) = origin airport index  
- \(j\) = destination airport index  
- \(k\) = aircraft type index  
- \(\text{seats}_{k}\) = seat capacity of aircraft type \(k\)  
- \(X_{i,j,k}\) = number of planes of type \(k\) assigned from \(i \to j\)  
- \(\text{demand}_{i,j}\) = passenger demand from \(i \to j\)  


In [32]:
#C3

# Aircraft Availability Here we are assuming we can run our aircraft for up to 20 hours in a day // normal opration time

# 20 * Σ X ijk ≥ (Freq ij)(Time ij)

for i in range(len(airports)):
    for j in range(len(airports)):
        prob += pulp.lpSum(20 * x[i][j][k] for k in range(len(aircrafts))) >= frequency[i][j] * time_flight[i][j],'Availability Constraint: {} to {}'.format(airports[i],airports[j])




**Aircraft Availability Constraint**

$$
20 \cdot \sum_{k} X_{i,j,k} \;\ge\; \text{frequency}_{i,j} \times \text{time}_{i,j}
$$

Where:
- \(i\) = origin airport index  
- \(j\) = destination airport index  
- \(k\) = aircraft type index  
- \(X_{i,j,k}\) = number of planes of type \(k\) assigned from \(i \to j\)  
- \(20\) = maximum operational hours per aircraft per day (assumption)  
- \(\text{frequency}_{i,j}\) = flights per day on route \(i \to j\)  
- \(\text{time}_{i,j}\) = flight duration in hours on route \(i \to j\)  


In [33]:
#C4 - Flight Frequency

# Ensures every route gets its required level of service.

# Σ Xijk ≥ frequency ij

for i in range(len(airports)):
    for j in range(len(airports)):
        prob += pulp.lpSum(x[i][j][k] for k in range(len(aircrafts))) >= frequency[i][j],'Frequency Constraint: {} to {}'.format(airports[i],airports[j])

**Flight Frequency Constraint**

$$
\sum_{k} X_{i,j,k} \;\ge\; \text{frequency}_{i,j}
$$

Where:
- \(i\) = origin airport index  
- \(j\) = destination airport index  
- \(k\) = aircraft type index  
- \(X_{i,j,k}\) = number of planes of type \(k\) assigned from \(i \to j\)  
- \(\text{frequency}_{i,j}\) = required flights per day on route \(i \to j\)  


In [ ]:
# Constraint - Part 5 – Number of aircrafts available

for k in range(len(aircrafts)):
    prob += pulp.lpSum(pulp.lpSum(x[i][j][k] for i in range(len(airports)))for j in range(len(airports))) <= aircrafts_avail[k],'Resource Constraint: {}'.format(aircrafts[k])

**Fleet Size Constraint**

$$
\sum_{i} \sum_{j} X_{i,j,k} \;\le\; \text{aircrafts\_avail}_{k}
$$

Where:
- \(i\) = origin airport index  
- \(j\) = destination airport index  
- \(k\) = aircraft type index  
- \(X_{i,j,k}\) = number of planes of type \(k\) assigned from \(i \to j\)  
- \(\text{aircrafts\_avail}_{k}\) = total number of planes of type \(k\) available in the fleet  


In [34]:
prob.solve()

1

In [35]:
from pulp import LpStatus

print("Status:", LpStatus[prob.status])  # Optimal, Infeasible, etc.
print("Objective value (Profit):", pulp.value(prob.objective))


Status: Optimal
Objective value (Profit): 2766333.0


In [36]:
for var in prob.variables():
    if var.varValue != 0:
        print(var.name, "=", var.varValue)


DEN_to_EWR,type_Airbus_320 = 5.0
DEN_to_EWR,type_Boeing_767 = 1.0
DEN_to_IAD,type_Airbus_320 = 4.0
DEN_to_IAH,type_Airbus_320 = 5.0
DEN_to_LAX,type_Boeing_787 = 5.0
DEN_to_ORD,type_Airbus_320 = 6.0
DEN_to_ORD,type_Boeing_787 = 3.0
DEN_to_SFO,type_Airbus_320 = 5.0
DEN_to_SFO,type_Boeing_767 = 1.0
EWR_to_DEN,type_Airbus_320 = 4.0
EWR_to_DEN,type_Boeing_787 = 1.0
EWR_to_IAD,type_Boeing_737 = 10.0
EWR_to_IAD,type_Boeing_787 = 1.0
EWR_to_IAH,type_Airbus_320 = 9.0
EWR_to_LAX,type_Airbus_320 = 14.0
EWR_to_ORD,type_Airbus_320 = 1.0
EWR_to_ORD,type_Boeing_737 = 5.0
EWR_to_ORD,type_Boeing_787 = 5.0
EWR_to_SFO,type_Airbus_320 = 5.0
EWR_to_SFO,type_Boeing_737 = 5.0
EWR_to_SFO,type_Boeing_787 = 2.0
IAD_to_DEN,type_Airbus_320 = 2.0
IAD_to_DEN,type_Boeing_767 = 1.0
IAD_to_EWR,type_Airbus_320 = 3.0
IAD_to_EWR,type_Boeing_737 = 5.0
IAD_to_EWR,type_Boeing_787 = 2.0
IAD_to_IAH,type_Airbus_320 = 2.0
IAD_to_IAH,type_Boeing_787 = 2.0
IAD_to_LAX,type_Airbus_320 = 5.0
IAD_to_LAX,type_Boeing_767 = 1.0
IAD_to_L

In [61]:
# Results
airport1=[]
airport2=[]
plane=[]
print("="*40,"\nSolution Status:", plp.LpStatus[prob.status])
print("Profit (Z) is: $", str(value(prob.objective)))
print("="*40)
for i in range(len(airports)):
    for j in range(len(airports)):
        for k in range(len(aircrafts)):
            if(x[i][j][k].varValue!=0):
                temp=x[i][j][k].varValue
                print('{} to {}: {} {}s'.format(airports[i],airports[j],temp,aircrafts[k]))
                airport1.append(airports[i])
                airport2.append(airports[j])
                plane.append(aircrafts[k])

df_paths = list(zip(airport1,airport2,plane))
df_paths = pd.DataFrame(df_paths,columns=['iata','iata','Type'])
df_paths_1 =pd.DataFrame(df_paths.iloc[:,0])
df_paths_2 =pd.DataFrame(df_paths.iloc[:,1])

Solution Status: Optimal
Profit (Z) is: $ 2766333.0
ORD to DEN: 1.0 Boeing 787s
ORD to DEN: 10.0 Boeing 737s
ORD to IAH: 5.0 Airbus 320s
ORD to LAX: 1.0 Boeing 787s
ORD to LAX: 5.0 Boeing 737s
ORD to EWR: 2.0 Boeing 787s
ORD to EWR: 1.0 Airbus 320s
ORD to EWR: 10.0 Boeing 737s
ORD to SFO: 2.0 Boeing 787s
ORD to SFO: 1.0 Airbus 320s
ORD to SFO: 1.0 Boeing 767s
ORD to IAD: 4.0 Airbus 320s
DEN to ORD: 3.0 Boeing 787s
DEN to ORD: 6.0 Airbus 320s
DEN to IAH: 5.0 Airbus 320s
DEN to LAX: 5.0 Boeing 787s
DEN to EWR: 5.0 Airbus 320s
DEN to EWR: 1.0 Boeing 767s
DEN to SFO: 5.0 Airbus 320s
DEN to SFO: 1.0 Boeing 767s
DEN to IAD: 4.0 Airbus 320s
IAH to ORD: 1.0 Boeing 787s
IAH to ORD: 3.0 Airbus 320s
IAH to DEN: 4.0 Airbus 320s
IAH to LAX: 10.0 Airbus 320s
IAH to EWR: 10.0 Airbus 320s
IAH to SFO: 5.0 Airbus 320s
IAH to IAD: 1.0 Airbus 320s
IAH to IAD: 5.0 Boeing 737s
LAX to ORD: 3.0 Boeing 787s
LAX to ORD: 2.0 Airbus 320s
LAX to DEN: 4.0 Boeing 787s
LAX to DEN: 1.0 Airbus 320s
LAX to DEN: 1.0 Boei

In [53]:
df_paths

,origin,destination,Type
0,ORD,DEN,Boeing 787
1,ORD,DEN,Boeing 737
2,ORD,IAH,Airbus 320
3,ORD,LAX,Boeing 787
4,ORD,LAX,Boeing 737
...,...,...,...
70,IAD,EWR,Boeing 787
71,IAD,EWR,Airbus 320
72,IAD,EWR,Boeing 737
73,IAD,SFO,Airbus 320


In [63]:
import plotly.graph_objects as go
import pandas as pd

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports[df_airports.country=="USA"]
hubs=df_airports[df_airports.iata.isin(airports)]

df_paths_1=df_paths_1.merge(df_airports,on='iata',how='left')
df_paths_2=df_paths_2.merge(df_airports,on='iata',how='left')

In [65]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = hubs['long'],
    lat = hubs['lat'],
    hoverinfo = 'text',
    text = hubs['airport'],
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))
flight_paths = []
for i in range(len(df_paths_1)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_paths_1['long'][i], df_paths_2['long'][i]],
            lat = [df_paths_1['lat'][i], df_paths_2['lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'blue'),
            opacity = float(df_paths_1['cnt'][i]) / float(df_paths_2['cnt'].max()),
        )
    )

fig.update_layout(
    title_text = 'ABC Airlines flight paths Project',
    showlegend = False,
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

In [72]:
# Extract results from LP variables
results = []
for i in range(len(airports)):
    for j in range(len(airports)):
        for k in range(len(aircrafts)):
            flights = x[i][j][k].varValue
            if flights > 0:
                results.append({
                    'Origin': airports[i],
                    'Destination': airports[j],
                    'Aircraft Type': aircrafts[k],
                    'Flights Assigned': flights,
                    'Seats per Flight': seat[k],
                    'Total Seats': flights * seat[k]
                })

df_results = pd.DataFrame(results)

df_results

,Origin,Destination,Aircraft Type,Flights Assigned,Seats per Flight,Total Seats
0,ORD,DEN,Boeing 787,1.0,300,300.0
1,ORD,DEN,Boeing 737,10.0,160,1600.0
2,ORD,IAH,Airbus 320,5.0,150,750.0
3,ORD,LAX,Boeing 787,1.0,300,300.0
4,ORD,LAX,Boeing 737,5.0,160,800.0
...,...,...,...,...,...,...
70,IAD,EWR,Boeing 787,2.0,300,600.0
71,IAD,EWR,Airbus 320,3.0,150,450.0
72,IAD,EWR,Boeing 737,5.0,160,800.0
73,IAD,SFO,Airbus 320,1.0,150,150.0


In [73]:
# If you want top N routes by flights
top_routes = df_results.groupby(['Origin', 'Destination'])['Flights Assigned'].sum().nlargest(10).reset_index()
print("Top 10 Routes by Flights:")
print(top_routes)


Top 10 Routes by Flights:
  Origin Destination  Flights Assigned
0    LAX         EWR              15.0
1    LAX         SFO              15.0
2    EWR         LAX              14.0
3    SFO         LAX              14.0
4    ORD         EWR              13.0
5    EWR         SFO              12.0
6    SFO         EWR              12.0
7    EWR         IAD              11.0
8    EWR         ORD              11.0
9    ORD         DEN              11.0


# Fleet Assignment – Integer Linear Program

## Parameters
- $ \text{cost}_{k} $ : operating cost per flight hour for aircraft type $k$ [$/\text{hour}$]  
- $ \text{avail}_{k} $ : number of available aircraft of type $k$ [units]  
- $ U $ : maximum usable hours per aircraft per day (assumed $U=20$) [hours/day]  

## Decision Variables
- $ X_{ijk} \in \mathbb{Z}_{\ge 0} $ : number of flights (or aircraft assignments) of type $k$ scheduled on route $i \to j$  

## Objective (maximize daily profit)
$$
\max Z =
\underbrace{\sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}}
\text{fare}_{ij} \cdot \text{demand}_{ij}}_{\text{Revenue (as modeled)}}
-
\underbrace{\sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} \sum_{k \in \mathcal{K}}
\text{cost}_{k} \cdot \text{time}_{ij} \cdot X_{ijk}}_{\text{Operating cost}}
$$

## Constraints

**(C1) No self-loops**  
$$
X_{iik} = 0 \quad \forall i \in \mathcal{I},\; k \in \mathcal{K}
$$

**(C2) Seat supply meets demand (allowing slack)**  
$$
\sum_{k \in \mathcal{K}} \text{seats}_{k} \cdot X_{ijk} \ge \text{demand}_{ij}
\quad \forall i \in \mathcal{I},\; j \in \mathcal{J}
$$

**(C3) Aircraft utilization per route (hours available ≥ hours required)**  
$$
U \cdot \sum_{k \in \mathcal{K}} X_{ijk} \ge \text{freq}_{ij} \cdot \text{time}_{ij}
\quad \forall i \in \mathcal{I},\; j \in \mathcal{J}
$$

**(C4) Frequency coverage**  
$$
\sum_{k \in \mathcal{K}} X_{ijk} \ge \text{freq}_{ij}
\quad \forall i \in \mathcal{I},\; j \in \mathcal{J}
$$

**(C5) Fleet size (by type)**  
$$
\sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} X_{ijk} \le \text{avail}_{k}
\quad \forall k \in \mathcal{K}
$$

## Variable Domains
$$
X_{ijk} \in \mathbb{Z}_{\ge 0} \quad \forall i \in \mathcal{I},\; j \in \mathcal{J},\; k \in \mathcal{K}
$$
